[1235. Maximum Profit in Job Scheduling](https://leetcode.com/problems/maximum-profit-in-job-scheduling/)

`endTime`是离散的，所以用`dict`来做的DP。没用`bisect`结果超时了

In [17]:
class Solution:
    def jobScheduling(self, startTime, endTime, profit) -> int:
        from collections import defaultdict
        job = sorted(zip(startTime, endTime, profit))
        
        end_at = defaultdict(int)
        for s, e, p in job:
            t = max((k for k in end_at if k <= s), default = 0, key = lambda k: end_at[k])
            end_at[e] = max(end_at[e], end_at[t] + p)
        print(end_at)
        
        return max(end_at.values())

In [18]:
Solution().jobScheduling(startTime = [1,2,3,3], endTime = [3,4,5,6], profit = [50,10,40,70])

defaultdict(<class 'int'>, {3: 50, 0: 0, 4: 10, 5: 90, 6: 120})


120

改了之后照旧，如果纯`bisect`不能确保`(e, profit)`的`profit`最大，用`max`就会TLE

In [69]:
class Solution:
    def jobScheduling(self, startTime, endTime, profit) -> int:
        from math import inf
        from bisect import bisect, insort
        
        job = sorted(zip(startTime, endTime, profit))
        dp = [(0, 0)]
        for s, e, p in job:
            i = bisect(dp, (s, inf))
            new = max((j[1] for j in dp[: i]), default = 0) + p
            insort(dp, (e, new))
            print(s, e, p, dp)
        # print(end_at)
        
        return max(i[1] for i in dp)

In [67]:
Solution().jobScheduling(startTime = [1,2,3,3], endTime = [3,4,5,6], profit = [50,10,40,70])

1 3 50 [(0, 0), (3, 50)]
2 4 10 [(0, 0), (3, 50), (4, 10)]
3 5 40 [(0, 0), (3, 50), (4, 10), (5, 90)]
3 6 70 [(0, 0), (3, 50), (4, 10), (5, 90), (6, 120)]


120

In [68]:
Solution().jobScheduling(
[4,2,4,8,2],
[5,5,5,10,8],
[1,2,8,10,4])

2 5 2 [(0, 0), (5, 2)]
2 8 4 [(0, 0), (5, 2), (8, 4)]
4 5 1 [(0, 0), (5, 1), (5, 2), (8, 4)]
4 5 8 [(0, 0), (5, 1), (5, 2), (5, 8), (8, 4)]
8 10 10 [(0, 0), (5, 1), (5, 2), (5, 8), (8, 4), (10, 18)]


18

向lee学习
- 不需要`inf`参与构建元素，只需要用`s + 1`搜索即可
- 还是像大部分interval题型用`endTime`来排序, 可以贪心地少存一些东西
    按`startTime`处理，我们不能简单的只对`dp[-1]`做比较
- 因为可以贪心地少存东西，就可以直接`append`而不用`insort`了

In [70]:
class Solution:
    def jobScheduling(self, startTime, endTime, profit) -> int:
        from bisect import bisect
        
        job = sorted(zip(startTime, endTime, profit), key = lambda x: x[1])
        dp = [(0, 0)]
        for s, e, p in job:
            i = bisect(dp, (s + 1, )) - 1
            new_p = dp[i][1] + p
            if new_p > dp[-1][1]:
                dp.append((e, new_p))
        
        return max(dp[-1][1])